In [1]:
from bert_serving.client import BertClient

In [2]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, BatchNormalization

Using TensorFlow backend.


Command line expressions for regular BERT and fine-tuned BERT alternatively. Adjust paths as needed.

In [3]:
#bert-serving-start -pooling_strategy NONE -max_seq_len 100 -model_dir G:/uncased_L-12_H-768_A-12 
#bert-serving-start -pooling_strategy NONE -max_seq_len 100 -model_dir G:/uncased_L-12_H-768_A-12 -tuned_model_dir G:/model_checkpoint  -ckpt_name=model.ckpt-2000

In [1]:
#To save time, only the dev set was used at some point. Testing on train and test set yielded the same results.
path_dev_tsv='../Data_preprocessed/dev.tsv'

In [2]:
from collections import Counter
def get_labels(path):
    df = pd.read_csv(path, sep='\t', index_col=0)
    labels = list(df['label'])
    print(Counter(labels))
    labels = np.asarray(labels)
    labels = to_categorical(labels)
    return labels

In [6]:
def get_texts(path):
    df = pd.read_csv(path, sep='\t', index_col=0)
    texts = list(df['text'])
    return texts

In [7]:
texts = get_texts(path_dev_tsv)

In [8]:
texts[:10]

['add me back you twat',
 'ruffled ntac eileen dahlia - beautiful color combination of pink orange yellow white a coll httptcoHdYEBvnZB',
 "i know bro nigga seen a ' african playing guard and was handling the hoe",
 'w left in the game are you retarded',
 "none of us went to these prestigious establishments so we're clearly not as superior as these fucking retards",
 "i called at least times to make my hair appointment and these bitches don't pick - - guess i have to stop in to make an appointment grrr",
 'i also like flip a bitch which is gabbys qualitywords',
 'why is that most of the niggas bitches i use to fw like to sub me face with tears of joy loudly crying face',
 'eee u sick fuck what do u even do oo politics you fuckin odd cunt',
 'you i was like you sneaky shit i saw him before i saw your dm because my phone is retarded']

In [9]:
bc = BertClient()

In [18]:
texts[1648] = '-' #tweet is empty

In [19]:
vecs = bc.encode(texts, show_tokens=True, is_tokenized=False) #is_tokenized=False, as many words were not found

c:\users\thora\appdata\local\programs\python\python36\lib\site-packages\bert_serving\client\__init__.py:294: UserWarning: "show_tokens=True", but the server does not support showing tokenization info to clients.
here is what you can do:
- start a new server with "bert-serving-start -show_tokens_to_client ..."
- or, use "encode(show_tokens=False)"
  warnings.warn('"show_tokens=True", but the server does not support showing tokenization info to clients.\n'


In [24]:
vecs.shape

(3000, 100, 768)

In [25]:
labels = get_labels(path_dev_tsv)

Counter({0: 1000, 2: 1000, 1: 1000})


In [27]:
from keras import Input, layers, Model
input_tensor= Input(shape=(100,768))

x = layers.LSTM(5)(input_tensor)
x=layers.Dropout(0.5)(x)
output_tensor=layers.Dense(3, activation='softmax')(x)

model2 = Model(input_tensor, output_tensor)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 768)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5)                 15480     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 18        
Total params: 15,498
Trainable params: 15,498
Non-trainable params: 0
_________________________________________________________________


In [28]:
model2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model2.fit(vecs, labels,
                            epochs=10,
                            validation_split=0.2)

Train on 2400 samples, validate on 600 samples
Epoch 1/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0989 - acc: 0.3292 - val_loss: 1.0987 - val_acc: 0.3283
Epoch 2/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0986 - acc: 0.3392 - val_loss: 1.0987 - val_acc: 0.3283
Epoch 3/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0990 - acc: 0.3379 - val_loss: 1.0988 - val_acc: 0.3283
Epoch 4/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0986 - acc: 0.3454 - val_loss: 1.0986 - val_acc: 0.3283
Epoch 5/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0989 - acc: 0.3296 - val_loss: 1.0986 - val_acc: 0.3283
Epoch 6/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0991 - acc: 0.3271 - val_loss: 1.0987 - val_acc: 0.3317
Epoch 7/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.0987 - acc: 0.3354 - val_loss: 1.0986 - val_acc: 0.3400
Epoch 8/10
2400